In [18]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
#from ed import * per importare ed senza bisogno di scrivere alcun prefisso
import ed

## Ex 7.1

a) As in Exercise 6.2, generate the ground state of the transverse field Ising model with
open boundary conditions for L = 14, g = 1.5, J ≡ 1. Make sure it is normalized to
⟨ψ|ψ⟩ = 1.

In [51]:
L = 14
g = 1.5
J = 1.
sx_list = ed.gen_sx_list(L)
sz_list = ed.gen_sz_list(L)
H = ed.gen_hamiltonian(sx_list, sz_list, g, J)
E, vecs = scipy.sparse.linalg.eigsh(H, which='SA')
psi0 = vecs[:, 0]
print(psi0)
np.linalg.norm(psi0)

[-8.14694335e-01 -8.41270756e-16 -1.33888127e-15 ... -2.31885084e-19
 -6.18242692e-18 -4.07427082e-06]


1.0000000000000016

b) Write a function compress(psi, L, chimax), which takes the state, the length of the
chain and the maximal desired bond dimension chimax as input and compresses the
state into MPS form using successive SVDs. It should return a list of L numpy
arrays, namely the M [n], each with 3 indices (αn, jn, αn+1).
Hint: Let us define the indices Rn = (jn, jn+1, . . . , jL), such that R1 ≡ i

In [76]:
def compress(psi, L, chimax):
    Ms = []
    psi = np.reshape(psi, (2, 2**(L-1)))
    n = 1
    lambda_n = [1]
    for i in range(1,L+1):
        psi = np.reshape(psi, (2*len(lambda_n),2**(L-i)))
        M_n, lambda_n, psitilde = np.linalg.svd(psi)

        keep = np.argsort ( lambda_n )[:: -1][: chimax ]
        M_n = M_n [: , keep ]
        lambda_n = lambda_n [ keep ]
        psitilde = psitilde [ keep , :]

        M_n = np.reshape(M_n, (n,2,len(lambda_n)))
        psi = lambda_n[:, np.newaxis] * psitilde[:, :]
        n = len(psitilde)

        Ms.append(M_n)
    
    return Ms
    



In [77]:
a = [1]
print(len(a))

1


c) What is the maximally necessary bond dimension for L = 14? Call compress() for
the ground state with χmax larger than that to get an exact MPS representation

The maximally necessary bond dimension is $2^{L/2} = 2^7$

In [78]:
chimax = 2**7
psiex = compress(psi0, 14, chimax)

In [79]:

entries_psiex = 0
for i in range(len(psiex)):
    Mi = np.array(psiex[i])
    a = np.shape(Mi)
    entries_psiex += a[0]*a[1]*a[2]


d) Call compress() again with χmax = 10 to get a compressed MPS. Compare
the number of floats stored in both MPS.
Hint: The number of elements in a numpy array M are given by M.size

In [80]:
psicompr = compress(psi0, 14, 10)

entries_psicompr = 0
for i in range(len(psicompr)):
    Mi = np.array(psicompr[i])
    a = np.shape(Mi)
    entries_psicompr += a[0]*a[1]*a[2]

In [81]:
print(entries_psicompr)
print(entries_psiex)

1688
43688


e) Write a function to calculate the overlap between two MPS. Recall from class that
there is an inefficient way (first contracting the bra and ket on top and bottom
separately and finally contracting over the j1, . . . jn) and an efficient way (contracting from left to right); implement the efficient one! Check that the overlap is (close to) 1 and calculate the overlap

In [82]:
def overlap(M1,M2):
    #M1 and M2 are the two sets of MPS of the two states
    #first element of each list is the matrix on the left
    N = len(M1)
    print(len(M1))
    print(len(M2))
    tensor2 = np.tensordot(M2[0].conj(), M1[0], [[0, 1], [0, 1]]) # [vL*] [i*] vR*, [vL] [i] vR -> vR* vR
    for i in range(1,N-1):
        tensor1 = np.tensordot(tensor2, M1[i], [1,0]) # vR* [vR], [vL] i vR -> vR* i vR
        tensor2 = np.tensordot(M2[i].conj(), tensor1, [[0,1],[0,1]]) # [vL*] [i*] vR*, [vR*] [i] vR -> vR* vR
    tensor1 = np.tensordot(tensor2, M1[N-1], [1,0]) # vR* [vR], [vL] i vR -> vR* i vR
    result = np.tensordot(M2[N-1].conj(), tensor1, [[0,1,2],[0,1,2]]) # [vL*] [i*] [vR*], [vR*] [i] [vR] -> [vR*] [vR]
    return result

In [84]:
print(overlap(psiex, psiex))
print()
print(overlap(psiex, psicompr))

14
14
1.0000000000000004

14
14
0.9999999999999892


f) Write the state |↑↑ · · · ↑⟩ as an MPS with bond dimension 1. Calculate the overlap
of this state with the ground state (using MPS techniques, i.e. use the function you
wrote in e) )

In [86]:
L = 14
g = 1.5
J = 0.
sx_list = ed.gen_sx_list(L)
sz_list = ed.gen_sz_list(L)
H = ed.gen_hamiltonian(sx_list, sz_list, g, J)
E, vecs = scipy.sparse.linalg.eigsh(H, which='SA')
psinew = vecs[:, 0]
print(psinew)
np.linalg.norm(psinew)

#ground state of Hamiltonian with J = 0 is exactly |↑↑ ·· ↑⟩
# which in comp basis is written as (1,0,0...0)

[-1.00000000e+00  1.69090273e-16  5.50885602e-16 ...  3.48033769e-17
 -5.20607496e-17 -7.52561939e-18]


1.0000000000000004

In [87]:
psinew_MPS = compress(psinew, 14, 1)

In [90]:
print("Overlap of psinew and groundstate: ", overlap(psinew_MPS, psiex))

14
14
Overlap of psinew and groundstate:  0.8146943347914242


## Ex 7.2